In [1]:
import numpy as np
import pandas as pd
import matplotlib as mlt
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import seaborn as sns
from sklearn.metrics import roc_curve, auc,accuracy_score,confusion_matrix,make_scorer
import collections
import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold,RandomizedSearchCV
from xgboost import XGBClassifier,plot_importance
from matplotlib import pyplot
from catboost import CatBoostClassifier

# Preprocessing of Data

In [2]:
def preprocess_data(train,test):

    combined = pd.concat([train,test])
    combined.loc[:, 'age2'] = combined.age * combined.age
    combined.loc[:, 'age3'] = combined.age * combined.age * combined.age
    combined.loc[:, 'HTHD'] = combined.heart_disease * combined.hypertension
    combined.loc[:, 'HTpHD'] = combined.heart_disease + combined.hypertension

    combined.loc[:, 'age_group'] = 'Low'
    combined.loc[combined.age < 40,'age_group'] = 'Children'
    combined.loc[combined.age < 40, 'age_group'] = 'Children'
    combined.loc[(combined.age >= 40) & (combined.age < 65), 'age_group'] = 'Adults'
    combined.loc[combined.age >= 65, 'age_group'] = 'Elderly'
    combined.loc[combined.age < 10,'smoking_status'] = 'never smoked'
    smoketab = pd.crosstab([combined.gender, np.round(combined.age, -1)], combined.smoking_status).apply(lambda x: x / x.sum(), axis=1)
    combined.smoking_status = combined.apply(lambda x: np.argmax(smoketab.loc[x['gender']].loc[int(np.round(x['age'],-1))]),axis=1)

    bmiByAge = combined[~combined.bmi.isnull()].pivot_table(index=np.round(combined[~combined.bmi.isnull()].age,-1),values='bmi',aggfunc='median')
    combined.loc[combined.bmi.isnull(),'bmi'] = combined[combined.bmi.isnull()].apply(lambda x: bmiByAge.loc[int(np.round(x['age'],-1))],axis=1)
    combined.loc[:, 'bmi2'] = combined.bmi * combined.bmi

    train,test = combined[:len(train)],combined[len(train):]
    return train,test

# One Hot Encoding

In [3]:
def OHE(train,test):
    combined = pd.concat([train,test])
    combined.Residence_type = LabelEncoder().fit_transform(combined.Residence_type)
    combined.smoking_status = LabelEncoder().fit_transform(combined.smoking_status)
    combined.age_group = LabelEncoder().fit_transform(combined.age_group)
    combined.work_type = LabelEncoder().fit_transform(combined.work_type)
    combined.HTpHD = LabelEncoder().fit_transform(combined.HTpHD)
    combined.gender = LabelEncoder().fit_transform(combined.gender)
    combined.ever_married = LabelEncoder().fit_transform(combined.ever_married)
    return combined[:len(train)],combined[len(train):]

# Logistic Regression

In [4]:
def LR(train,test,params):
    X_Train,X_Val,Y_Train,Y_Val = train_test_split(train,train.stroke,random_state=101)
    model = LogisticRegression(class_weight='balanced',random_state=101)
    model.fit(X_Train[params],Y_Train)

    prediction = model.predict_proba(X_Val[params])
    prediction = prediction[:,1]
    print('Validation Accuracy: ')
    fpr,tpr,_ = roc_curve(Y_Val, prediction)
    print(auc(fpr,tpr))

    prediction = model.predict_proba(X_Train[params])
    prediction = prediction[:, 1]
    print('Train Accuracy: ')
    fpr, tpr, _ = roc_curve(Y_Train, prediction)
    print(auc(fpr, tpr))

    model = LogisticRegression(class_weight='balanced',random_state=101)
    model.fit(train[params], train.stroke)
    prediction = model.predict_proba(test[params])
    prediction = prediction[:,1]
    test.loc[:,'stroke'] = prediction
    test[['id','stroke']].to_csv(path+'/output_lr.csv',index=False)

# XGB Classifier

In [5]:
def XGB(train,test,params):
    parameters = {
        'min_child_weight': [5, 10],
        'gamma': [0.5, 2, 5],
        'subsample': [0.4, 0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5]
    }
    folds = 5
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1001)
    model = XGBClassifier(n_estimators=100, learning_rate=0.02, objective='binary:logistic',
                          silent=True, nthread=1)

    # define scoring function
    def custom_auc(ground_truth, predictions):
        # I need only one column of predictions["0" and "1"]. You can get an error here
        # while trying to return both columns at once
        fpr, tpr, _ = roc_curve(ground_truth, predictions[:, 1], pos_label=1)
        return auc(fpr, tpr)

        # to be standart sklearn's scorer

    my_auc = make_scorer(custom_auc, greater_is_better=True, needs_proba=True)

    search = RandomizedSearchCV(model, parameters,
                          cv=skf.split(train[params], train.stroke),scoring=my_auc,n_iter=15)
    search.fit(train[params],train.stroke)

    print('Best Score: ' + str(search.best_score_))
    print('Best Params: ')
    print(search.best_params_)

    prediction = search.predict_proba(test[params])
    prediction = prediction[:, 1]
    test.loc[:,'stroke'] = prediction
    test[['id','stroke']].to_csv(path+'/output_xgb.csv',index=False)

# CatBoost Classifier

In [6]:
def CB(train,test,params,idx):

    parameters = {
        'iterations': [20,50,100],
        'learning_rate': [0.1, 0.01, 0.001],
        'depth': [3, 5, 7],
    }
    folds = 5
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1001)
    model = CatBoostClassifier(loss_function='Logloss')

    # define scoring function
    def custom_auc(ground_truth, predictions):
        # I need only one column of predictions["0" and "1"]. You can get an error here
        # while trying to return both columns at once
        fpr, tpr, _ = roc_curve(ground_truth, predictions[:, 1], pos_label=1)
        return auc(fpr, tpr)

        # to be standart sklearn's scorer

    my_auc = make_scorer(custom_auc, greater_is_better=True, needs_proba=True)

    search = RandomizedSearchCV(model, parameters,
                          cv=skf.split(train[params], train.stroke),scoring=my_auc,n_iter=15)
    search.fit(train[params],train.stroke,cat_features=idx)

    print('Best Score: ' + str(search.best_score_))
    print('Best Params: ')
    print(search.best_params_)

    prediction = search.predict_proba(test[params])
    prediction = prediction[:, 1]
    test.loc[:,'stroke'] = prediction
    test[['id','stroke']].to_csv(path+'/output_cb.csv',index=False)

# Data Loading and Predicitons

In [7]:
path = "C:/Users/gauta/Desktop/AV McKinsey April 2018"
train = pd.read_csv(path+'/train.csv')
test = pd.read_csv(path+'/test.csv')
train,test = preprocess_data(train,test)

CB(train,test,['age','heart_disease','hypertension','HTpHD','avg_glucose_level','gender','smoking_status'],[1,2,3,5,6])

train,test = OHE(train,test)
LR(train,test,['smoking_status','age','heart_disease','hypertension','HTpHD','avg_glucose_level','gender'])
XGB(train,test,['smoking_status','age','heart_disease','hypertension','HTpHD','avg_glucose_level','gender'])


cbPredictions = pd.read_csv(path+'/output_cb.csv').stroke
lrPredictions = pd.read_csv(path+'/output_lr.csv').stroke
xgbPredictions = pd.read_csv(path+'/output_xgb.csv').stroke
finalPrediction = (cbPredictions + lrPredictions + xgbPredictions)/3.0
test.loc[:,'stroke'] = finalPrediction
test[['id','stroke']].to_csv(path+'/final_output.csv',index=False)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:52: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


0:	learn: 0.6910376	total: 193ms	remaining: 19.1s
1:	learn: 0.6888562	total: 245ms	remaining: 12s
2:	learn: 0.6868902	total: 270ms	remaining: 8.74s
3:	learn: 0.6848255	total: 308ms	remaining: 7.39s
4:	learn: 0.6828815	total: 329ms	remaining: 6.25s
5:	learn: 0.6808778	total: 354ms	remaining: 5.54s
6:	learn: 0.6786680	total: 393ms	remaining: 5.21s
7:	learn: 0.6766720	total: 428ms	remaining: 4.92s
8:	learn: 0.6746824	total: 449ms	remaining: 4.54s
9:	learn: 0.6722627	total: 482ms	remaining: 4.34s
10:	learn: 0.6697262	total: 526ms	remaining: 4.25s
11:	learn: 0.6677648	total: 547ms	remaining: 4.01s
12:	learn: 0.6658040	total: 567ms	remaining: 3.8s
13:	learn: 0.6635556	total: 611ms	remaining: 3.75s
14:	learn: 0.6616078	total: 639ms	remaining: 3.62s
15:	learn: 0.6596670	total: 658ms	remaining: 3.45s
16:	learn: 0.6574957	total: 687ms	remaining: 3.35s
17:	learn: 0.6551788	total: 720ms	remaining: 3.28s
18:	learn: 0.6531981	total: 743ms	remaining: 3.17s
19:	learn: 0.6513533	total: 760ms	remaining:

65:	learn: 0.5640309	total: 2.16s	remaining: 1.11s
66:	learn: 0.5623719	total: 2.19s	remaining: 1.08s
67:	learn: 0.5605421	total: 2.23s	remaining: 1.05s
68:	learn: 0.5589253	total: 2.25s	remaining: 1.01s
69:	learn: 0.5572764	total: 2.28s	remaining: 977ms
70:	learn: 0.5555347	total: 2.31s	remaining: 944ms
71:	learn: 0.5539820	total: 2.33s	remaining: 907ms
72:	learn: 0.5524345	total: 2.36s	remaining: 872ms
73:	learn: 0.5507756	total: 2.41s	remaining: 845ms
74:	learn: 0.5492561	total: 2.43s	remaining: 809ms
75:	learn: 0.5476302	total: 2.46s	remaining: 777ms
76:	learn: 0.5460442	total: 2.5s	remaining: 746ms
77:	learn: 0.5444398	total: 2.53s	remaining: 715ms
78:	learn: 0.5428736	total: 2.58s	remaining: 685ms
79:	learn: 0.5412970	total: 2.6s	remaining: 649ms
80:	learn: 0.5397276	total: 2.63s	remaining: 616ms
81:	learn: 0.5382273	total: 2.65s	remaining: 581ms
82:	learn: 0.5367309	total: 2.67s	remaining: 547ms
83:	learn: 0.5352406	total: 2.69s	remaining: 512ms
84:	learn: 0.5334201	total: 2.72s

29:	learn: 0.6310555	total: 932ms	remaining: 2.17s
30:	learn: 0.6292536	total: 952ms	remaining: 2.12s
31:	learn: 0.6274087	total: 988ms	remaining: 2.1s
32:	learn: 0.6253685	total: 1.02s	remaining: 2.08s
33:	learn: 0.6234705	total: 1.06s	remaining: 2.05s
34:	learn: 0.6212050	total: 1.1s	remaining: 2.04s
35:	learn: 0.6194507	total: 1.11s	remaining: 1.98s
36:	learn: 0.6176096	total: 1.14s	remaining: 1.94s
37:	learn: 0.6158478	total: 1.16s	remaining: 1.89s
38:	learn: 0.6140722	total: 1.18s	remaining: 1.85s
39:	learn: 0.6119723	total: 1.2s	remaining: 1.8s
40:	learn: 0.6102351	total: 1.22s	remaining: 1.76s
41:	learn: 0.6082676	total: 1.25s	remaining: 1.73s
42:	learn: 0.6065170	total: 1.28s	remaining: 1.69s
43:	learn: 0.6047193	total: 1.3s	remaining: 1.66s
44:	learn: 0.6026219	total: 1.35s	remaining: 1.65s
45:	learn: 0.6009275	total: 1.36s	remaining: 1.6s
46:	learn: 0.5991259	total: 1.4s	remaining: 1.58s
47:	learn: 0.5974054	total: 1.44s	remaining: 1.56s
48:	learn: 0.5954960	total: 1.46s	rema

94:	learn: 0.5161067	total: 3.16s	remaining: 166ms
95:	learn: 0.5146063	total: 3.21s	remaining: 134ms
96:	learn: 0.5131579	total: 3.23s	remaining: 99.9ms
97:	learn: 0.5116967	total: 3.26s	remaining: 66.5ms
98:	learn: 0.5101689	total: 3.29s	remaining: 33.3ms
99:	learn: 0.5083649	total: 3.33s	remaining: 0us
0:	learn: 0.6910710	total: 25.5ms	remaining: 2.52s
1:	learn: 0.6886166	total: 55ms	remaining: 2.7s
2:	learn: 0.6863038	total: 86.3ms	remaining: 2.79s
3:	learn: 0.6842998	total: 113ms	remaining: 2.71s
4:	learn: 0.6823008	total: 137ms	remaining: 2.6s
5:	learn: 0.6801679	total: 162ms	remaining: 2.54s
6:	learn: 0.6781988	total: 191ms	remaining: 2.54s
7:	learn: 0.6760610	total: 216ms	remaining: 2.49s
8:	learn: 0.6741233	total: 244ms	remaining: 2.46s
9:	learn: 0.6721550	total: 269ms	remaining: 2.42s
10:	learn: 0.6699138	total: 294ms	remaining: 2.38s
11:	learn: 0.6679447	total: 320ms	remaining: 2.35s
12:	learn: 0.6659365	total: 344ms	remaining: 2.31s
13:	learn: 0.6640010	total: 367ms	remaini

...

9:	learn: 0.1025681	total: 227ms	remaining: 227ms
10:	learn: 0.0957828	total: 255ms	remaining: 208ms
11:	learn: 0.0921909	total: 275ms	remaining: 183ms
12:	learn: 0.0879925	total: 296ms	remaining: 159ms
13:	learn: 0.0861904	total: 315ms	remaining: 135ms
14:	learn: 0.0842824	total: 340ms	remaining: 113ms
15:	learn: 0.0826101	total: 364ms	remaining: 91.1ms
16:	learn: 0.0813268	total: 394ms	remaining: 69.5ms
17:	learn: 0.0799553	total: 420ms	remaining: 46.7ms
18:	learn: 0.0790441	total: 452ms	remaining: 23.8ms
19:	learn: 0.0785091	total: 475ms	remaining: 0us
0:	learn: 0.5068363	total: 26ms	remaining: 493ms
1:	learn: 0.3627734	total: 50.8ms	remaining: 457ms
2:	learn: 0.2704700	total: 75.3ms	remaining: 427ms
3:	learn: 0.2095512	total: 99.7ms	remaining: 399ms
4:	learn: 0.1678647	total: 124ms	remaining: 372ms
5:	learn: 0.1419576	total: 149ms	remaining: 347ms
6:	learn: 0.1247244	total: 172ms	remaining: 320ms
7:	learn: 0.1124167	total: 196ms	remaining: 294ms
8:	learn: 0.1033617	total: 218ms	rem

36:	learn: 0.2512418	total: 832ms	remaining: 292ms
37:	learn: 0.2453997	total: 858ms	remaining: 271ms
38:	learn: 0.2404331	total: 883ms	remaining: 249ms
39:	learn: 0.2343785	total: 906ms	remaining: 226ms
40:	learn: 0.2298020	total: 926ms	remaining: 203ms
41:	learn: 0.2250222	total: 951ms	remaining: 181ms
42:	learn: 0.2198631	total: 973ms	remaining: 158ms
43:	learn: 0.2155645	total: 997ms	remaining: 136ms
44:	learn: 0.2116957	total: 1.01s	remaining: 113ms
45:	learn: 0.2062484	total: 1.04s	remaining: 90.4ms
46:	learn: 0.2024887	total: 1.06s	remaining: 67.9ms
47:	learn: 0.1978386	total: 1.09s	remaining: 45.3ms
48:	learn: 0.1934907	total: 1.11s	remaining: 22.6ms
49:	learn: 0.1901813	total: 1.13s	remaining: 0us
0:	learn: 0.6729319	total: 28ms	remaining: 1.37s
1:	learn: 0.6539354	total: 55.1ms	remaining: 1.32s
2:	learn: 0.6353962	total: 79.9ms	remaining: 1.25s
3:	learn: 0.6158586	total: 104ms	remaining: 1.19s
4:	learn: 0.5965632	total: 129ms	remaining: 1.16s
5:	learn: 0.5794763	total: 154ms	

0:	learn: 0.6910931	total: 26.9ms	remaining: 1.32s
1:	learn: 0.6887961	total: 83.6ms	remaining: 2.01s
2:	learn: 0.6863079	total: 124ms	remaining: 1.95s
3:	learn: 0.6842758	total: 154ms	remaining: 1.77s
4:	learn: 0.6821910	total: 203ms	remaining: 1.82s
5:	learn: 0.6800135	total: 233ms	remaining: 1.71s
6:	learn: 0.6780018	total: 257ms	remaining: 1.58s
7:	learn: 0.6759898	total: 294ms	remaining: 1.54s
8:	learn: 0.6740010	total: 314ms	remaining: 1.43s
9:	learn: 0.6719430	total: 342ms	remaining: 1.37s
10:	learn: 0.6699122	total: 366ms	remaining: 1.3s
11:	learn: 0.6679330	total: 389ms	remaining: 1.23s
12:	learn: 0.6655904	total: 409ms	remaining: 1.16s
13:	learn: 0.6635807	total: 433ms	remaining: 1.11s
14:	learn: 0.6613084	total: 476ms	remaining: 1.11s
15:	learn: 0.6590489	total: 499ms	remaining: 1.06s
16:	learn: 0.6569117	total: 536ms	remaining: 1.04s
17:	learn: 0.6549750	total: 556ms	remaining: 989ms
18:	learn: 0.6531173	total: 579ms	remaining: 944ms
19:	learn: 0.6508392	total: 611ms	remain

13:	learn: 0.6630927	total: 468ms	remaining: 1.2s
14:	learn: 0.6611866	total: 487ms	remaining: 1.14s
15:	learn: 0.6590692	total: 516ms	remaining: 1.09s
16:	learn: 0.6571098	total: 549ms	remaining: 1.06s
17:	learn: 0.6552270	total: 569ms	remaining: 1.01s
18:	learn: 0.6531340	total: 613ms	remaining: 1s
19:	learn: 0.6512051	total: 641ms	remaining: 961ms
20:	learn: 0.6492936	total: 675ms	remaining: 932ms
21:	learn: 0.6472516	total: 717ms	remaining: 913ms
22:	learn: 0.6449023	total: 766ms	remaining: 899ms
23:	learn: 0.6430198	total: 789ms	remaining: 855ms
24:	learn: 0.6412001	total: 818ms	remaining: 818ms
25:	learn: 0.6389739	total: 868ms	remaining: 802ms
26:	learn: 0.6368783	total: 933ms	remaining: 795ms
27:	learn: 0.6347243	total: 985ms	remaining: 774ms
28:	learn: 0.6325330	total: 1.01s	remaining: 736ms
29:	learn: 0.6306863	total: 1.04s	remaining: 695ms
30:	learn: 0.6286454	total: 1.07s	remaining: 655ms
31:	learn: 0.6263083	total: 1.12s	remaining: 628ms
32:	learn: 0.6244820	total: 1.14s	r

31:	learn: 0.0745704	total: 879ms	remaining: 495ms
32:	learn: 0.0744576	total: 918ms	remaining: 473ms
33:	learn: 0.0744332	total: 948ms	remaining: 446ms
34:	learn: 0.0743827	total: 976ms	remaining: 418ms
35:	learn: 0.0743514	total: 1s	remaining: 391ms
36:	learn: 0.0743120	total: 1.03s	remaining: 363ms
37:	learn: 0.0742459	total: 1.06s	remaining: 336ms
38:	learn: 0.0741993	total: 1.09s	remaining: 307ms
39:	learn: 0.0741971	total: 1.11s	remaining: 278ms
40:	learn: 0.0741921	total: 1.13s	remaining: 248ms
41:	learn: 0.0741409	total: 1.15s	remaining: 220ms
42:	learn: 0.0740600	total: 1.18s	remaining: 191ms
43:	learn: 0.0740248	total: 1.2s	remaining: 164ms
44:	learn: 0.0740244	total: 1.22s	remaining: 135ms
45:	learn: 0.0740092	total: 1.24s	remaining: 108ms
46:	learn: 0.0739903	total: 1.26s	remaining: 80.5ms
47:	learn: 0.0739699	total: 1.28s	remaining: 53.3ms
48:	learn: 0.0739398	total: 1.3s	remaining: 26.6ms
49:	learn: 0.0739222	total: 1.33s	remaining: 0us
Best Score: 0.8502501398366135
Best